## Scrape Validation Dataset (2015 movies)

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import re
from tqdm import tqdm
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
newlist = pd.read_csv("new_movies.csv")
newlist = newlist.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
newlist.head()

,titles_new,wiki_url_new,imdb_link_new,rt_link_new,imdb_id
0,Pinnokam,/wiki/Pinnokam,NaN,NaN,NaN
1,20 Once Again,/wiki/20_Once_Again,http://www.imdb.com/title/tt4344878/,NaN,tt4344878
2,Blackhat,/wiki/Blackhat_(film),http://www.imdb.com/title/tt2717822/,http://www.rottentomatoes.com/m/blackhat/,tt2717822
3,I,/wiki/I_(film),http://www.imdb.com/title/tt2302966/,NaN,tt2302966
4,Wild Card,/wiki/Wild_Card_(2015_film),http://www.imdb.com/title/tt2231253/,http://www.rottentomatoes.com/m/heat_2013/,tt2231253


In [3]:
newlist = newlist[newlist.imdb_id.isnull()!=1]
print "Number of new movies in 2015:", len(newlist)
#newlist[1:10]

Number of new movies in 2015: 305


In [4]:
### Need to run this block of code before reading them all from api...
import omdb
import json
trash = []
req = omdb.request(i=newlist.imdb_id.values[1], type='movie', tomatoes=True)
temp = json.loads(req.text)
trash.append(temp)
req = omdb.request(i=newlist.imdb_id.values[2], type='movie', tomatoes=True)
temp = json.loads(req.text)
trash.append(temp)
len(trash)
pd.DataFrame(trash)

,Actors,Awards,BoxOffice,Country,DVD,Director,Genre,Language,Metascore,Plot,Poster,Production,Rated,Released,Response,Runtime,Title,Type,Website,Writer,Year,imdbID,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoImage,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews
0,"Chris Hemsworth, Leehom Wang, Wei Tang, Viola ...",1 nomination.,$7.8M,USA,12 May 2015,Michael Mann,"Action, Crime, Drama","English, Mandarin, Spanish",51,A furloughed convict and his American and Chin...,http://ia.media-imdb.com/images/M/MV5BMTg1NDUy...,Universal Studios,R,16 Jan 2015,True,133 min,Blackhat,movie,http://www.blackhatthemovie.com/,Morgan Davis Foehl,2015,tt2717822,5.4,"32,017","Thematically timely but dramatically inert, Bl...",53,rotten,34,4.9,157,104,24,2.5,19310
1,"'Chiyaan' Vikram, Amy Jackson, Suresh Gopi, Up...",N/A,N/A,India,N/A,S. Shankar,"Action, Romance, Thriller","Tamil, Hindi, Telugu",N/A,"Lingesan, a deformed hunchback who was once a ...",http://ia.media-imdb.com/images/M/MV5BMTQyNDk3...,Aascar Films,N/A,14 Jan 2015,True,170 min,i,movie,N/A,"A.N. Balakrishnan (dialogue), S. Shankar (stor...",2015,tt2302966,7.7,"11,351",N/A,N/A,N/A,N/A,N/A,N/A,N/A,55,3.3,139


In [5]:
import requests
import json

movieslist = []
movienotfound = []
i = 0
for t in tqdm(newlist.imdb_id.values, leave=True):
    try:
        time.sleep(0.5)
        #url = "http://www.omdbapi.com/?t={}&y=&plot=full&r=json".format(t)
        #temp = json.loads(requests.get(url).text)
        req = omdb.request(i=t, type='movie', tomatoes=True)
        temp = json.loads(req.text)
        if str(temp['Response'])== 'False':
            movienotfound.append(t)
        if str(temp['Response'])== 'True':
            movieslist.append(temp)
    except: 
        movienotfound.append(t)      

        pass

100%|██████████| 305/305 [02:40<00:00,  1.92it/s]


In [6]:
print "Number of movies not found:", len(movieslist) - len(newlist)

Number of movies not found: 0


In [7]:
movies = pd.DataFrame(movieslist)
#print movies.Awards[1]
print "Make sure all the movie type is 'movie':", movies[movies.Type!='movie'].shape[0]
print "Make sure all the movie is found in the api portal:", movies[movies.Response=='False'].shape[0]

## remove games, series and NA types.
movies = movies[movies.Type=='movie']
movies = movies[movies.Response=='True']

def runtime(s):
    ss = s.split()
    if len(ss) > 1 and int(ss[0]) > 60:
        return int(ss[0])
    else:
        return None
    
movies['Runtime'] = movies.apply(lambda r: runtime(r.Runtime), axis=1)

Make sure all the movie type is 'movie': 0
Make sure all the movie is found in the api portal: 0


We need to extract the correspondeing information from award and put them into individual columns.

In [8]:
def numNomiOscar(s):
    re1='(Nominated)'
    re2='.*?'
    re3='(\\d+)'
    re4='.*?'
    re5='(Oscar)'
    re6='.*?'

    rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
    
    m = rg.search(s)
    if m:
        return int(m.group(2))
    else:
        return 0

def numWonOscar(s):
    re1='(Won)'
    re2='.*?'
    re3='(\\d+)'
    re4='.*?'
    re5='(Oscar)'
    re6='.*?'
    
    rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
    m = rg.search(s)
    if m:
        return int(m.group(2))
    else:
        return 0
    
def anotherWins(s):
    re1='(Another)'
    re2='.*?'
    re3='(\\d+)'
    re4='.*?'
    re5='(win)'
    re6='.*?'
    
    rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
    m = rg.search(s)
    if m:
        return int(m.group(2))
    else:
        return 0

def anotherNoms(s):
    re1='.*?'	# Non-greedy match on filler
    re2='\\d+'	# Uninteresting: int
    re3='.*?'	# Non-greedy match on filler
    re4='\\d+'	# Uninteresting: int
    re5='.*?'	# Non-greedy match on filler
    re6='(\\d+)'	# Integer Number 1
    re7='.*?'	# Non-greedy match on filler
    re8='(nomination)'	# Word 1

    rg = re.compile(re1+re2+re3+re4+re5+re6+re7+re8,re.IGNORECASE|re.DOTALL)
    m = rg.search(s)
    if m:
        return int(m.group(1))
    else:
        return 0


In [9]:
movies['Nominated Oscars'] = movies.apply(lambda r: numNomiOscar(r.Awards), axis=1)
movies['Won Oscars'] = movies.apply(lambda r: numWonOscar(r.Awards), axis=1)
movies['Other wins'] = movies.apply(lambda r: anotherWins(r.Awards), axis=1)
movies['Other nominations'] = movies.apply(lambda r: anotherNoms(r.Awards), axis=1)
import datetime
from dateutil import parser
movies['Released'] = movies.apply(lambda r: parser.parse(r.Released) if r.Released != 'N/A' else False, axis=1)
movies = movies.drop(['Plot', 'Poster', 'Writer', 'Type', 'Response'], axis=1)
movies = movies
#cols = list(movies.columns.values)
#print cols
movies = movies[[u'imdbID', u'Title', u'Released', u'Awards', u'Director', u'Actors', u'Country', u'Genre', u'Language',
                 u'Rated', u'Runtime', u'Year', u'Metascore', u'imdbRating', u'imdbVotes', u'tomatoConsensus', u'tomatoFresh',
                 u'tomatoMeter', u'tomatoRating', u'tomatoReviews', u'tomatoRotten', u'tomatoUserMeter',
                 u'tomatoUserRating', u'tomatoUserReviews', 'Nominated Oscars', 'Won Oscars', 'Other wins', 'Other nominations']]  

Note that the **Actors** and **Director** field may contain more than one person, and soon we would like to incorprate information of actors and directors to build new quantitative feature to measure the qualaity of a movie, so it will be more handy if we could explode the actor list and director list so that we would have multiple rows of under the same movie title but the actor and director field is different.

In [10]:
rows = []
_ = movies.apply(lambda row: [rows.append([row['imdbID'], row['Title'], row['Released'], row['Awards'], row['Director'],
                                           nn.strip(), row['Country'], row['Language'], row[u'Genre'],row['Rated'],
                                           row['Runtime'], row['Year'], row['Metascore'], row['imdbRating'],
                                           row['imdbVotes'], row['tomatoConsensus'], row['tomatoFresh'], row['tomatoMeter'],
                                           row['tomatoRating'], row['tomatoReviews'], row['tomatoRotten'], row['tomatoUserMeter'],
                                           row['tomatoUserRating'], row['tomatoUserReviews'], row['Nominated Oscars'],
                                           row['Won Oscars'], row['Other wins'], row['Other nominations']]) 
                         for nn in row.Actors.strip().split(',')], axis=1)
#rows
movies = pd.DataFrame(rows, columns=movies.columns)
movies.head()

,imdbID,Title,Released,Awards,Director,Actors,Country,Genre,Language,Rated,Runtime,Year,Metascore,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,Nominated Oscars,Won Oscars,Other wins,Other nominations
0,tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Zishan Yang,"China, South Korea",Mandarin,Comedy,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
1,tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Ya-Lei Kuei,"China, South Korea",Mandarin,Comedy,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
2,tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Bo-lin Chen,"China, South Korea",Mandarin,Comedy,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
3,tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Deshun Wang,"China, South Korea",Mandarin,Comedy,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
4,tt2717822,Blackhat,2015-01-16 00:00:00,1 nomination.,Michael Mann,Chris Hemsworth,USA,"English, Mandarin, Spanish","Action, Crime, Drama",R,133,2015,51,5.4,"32,017","Thematically timely but dramatically inert, Bl...",53,34,4.9,157,104,24,2.5,19310,0,0,0,0


In [11]:
rows = []
_ = movies.apply(lambda row: [rows.append([row['imdbID'], row['Title'], row['Released'], row['Awards'], nn.strip(),
                                           row['Actors'], row['Country'], row['Language'], row[u'Genre'],row['Rated'],
                                           row['Runtime'], row['Year'], row['Metascore'], row['imdbRating'],
                                           row['imdbVotes'], row['tomatoConsensus'], row['tomatoFresh'], row['tomatoMeter'],
                                           row['tomatoRating'], row['tomatoReviews'], row['tomatoRotten'], row['tomatoUserMeter'],
                                           row['tomatoUserRating'], row['tomatoUserReviews'], row['Nominated Oscars'],row['Won Oscars'],
                                           row['Other wins'], row['Other nominations']]) 
                         for nn in row.Director.strip().split(',')], axis=1)
#rows
movies = pd.DataFrame(rows, columns=movies.columns).set_index('imdbID')
movies.head()

,Title,Released,Awards,Director,Actors,Country,Genre,Language,Rated,Runtime,Year,Metascore,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,Nominated Oscars,Won Oscars,Other wins,Other nominations
imdbID,,,,,,,,,,,,,,,,,,,,,,,,,,,
tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Zishan Yang,"China, South Korea",Comedy,Mandarin,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Ya-Lei Kuei,"China, South Korea",Comedy,Mandarin,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Bo-lin Chen,"China, South Korea",Comedy,Mandarin,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
tt4344878,Miss Granny,2015-01-16 00:00:00,1 win & 2 nominations.,Leste Chen,Deshun Wang,"China, South Korea",Comedy,Mandarin,N/A,131,2015,N/A,6.8,473,N/A,N/A,N/A,N/A,N/A,N/A,89,4.3,86,0,0,0,0
tt2717822,Blackhat,2015-01-16 00:00:00,1 nomination.,Michael Mann,Chris Hemsworth,USA,"Action, Crime, Drama","English, Mandarin, Spanish",R,133,2015,51,5.4,"32,017","Thematically timely but dramatically inert, Bl...",53,34,4.9,157,104,24,2.5,19310,0,0,0,0


We then explode the **Genre** list as we did for homework 1, and create an indicator variable for each genre.

In [12]:
Genre = set()
for m in movies.Genre:
    Genre.update(g.strip() for g in str(m).split(',') if g != '')
Genre = sorted(Genre)
#make a column for each genre
for genre in Genre:
    movies[genre] = [genre.strip() in str(m).split(', ') for m in movies.Genre]
movies = movies.drop('Genre', axis=1)
movies = movies.sort_index(axis=0)

movies.head(5)

,Title,Released,Awards,Director,Actors,Country,Language,Rated,Runtime,Year,Metascore,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,Nominated Oscars,Won Oscars,Other wins,Other nominations,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,N/A,Romance,Sci-Fi,Sport,Thriller,War,Western
imdbID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Chris Pratt,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Irrfan Khan,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Vincent D'Onofrio,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Bryce Dallas Howard,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
tt0470752,Ex Machina,2015-04-24 00:00:00,2 wins & 7 nominations.,Alex Garland,Domhnall Gleeson,UK,English,R,108,2015,78,7.7,"194,256",Ex Machina leans heavier on ideas than effects...,199,92,8.0,216,17,85,4.0,61929,0,0,0,0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False


In [13]:
movies.shape

(1304, 48)

## Parse 2015 movies for prediction. 

In [18]:
newmovies = movies
#newmovies.columns

In [19]:
tts = newmovies.index
tts2 = []
for t in tts:
    if str(t)!='nan':
        tts2.append(str(t))
tts = tts2

In [23]:
import bs4
from bs4 import BeautifulSoup
%time
directorlist = []
for t in tqdm(tts,leave=True):
    url = "http://www.imdb.com/title/{}/".format(t)
    t1970 = requests.get(url)
    soup = BeautifulSoup(t1970.text, "html.parser")
    tb = soup.find("table").findAll("div", attrs = {'class','txt-block'})
    if tb[0].find('a')['href'] == None:
        directorlist.append(None)
    else:
        directorlist.append(tb[0].find('a')['href'])

100%|██████████| 1304/1304 [12:53<00:00,  1.87it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [24]:
#directorlist2 = dict(zip(tts, directorlist)) 
#fd=open("directorlist2015.json","w")
#json.dump(directorlist, fd)
#fd.close()

In [25]:
#with open("directorlist2015.json") as json_file:
#    directorlist = json.load(json_file)

In [26]:
%%time
director_dict = {"Name":[], "DOB":[], "Birthplace":[], "wins":[], "credits":[]}

for u in tqdm(directorlist, leave=True):
    #print u
    url = 'http://www.imdb.com{}'.format(u)
    t1990=requests.get(url)
    soup=BeautifulSoup(t1990.text, "html.parser")
    try:
        divs=soup.findAll('div', attrs = {'id':'name-born-info'})[0]
        Name = soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].find('span',attrs={'class':'itemprop'}).get_text()
        try: 
            DOB=divs.find('time').get('datetime')
        except:
            DOB=None 
        try:
            Birthplace=divs.findAll('a')[-1].get_text()
        except:
            Birthplace=None
        try:
            wins=soup.findAll('div', attrs = {'class':'article highlighted'})[0].find_all('span')[0].get_text().strip()
        except:
            wins=0
        try:
            credits=soup.findAll('div', attrs = {'id':'filmo-head-director'})[0].get_text().strip()[22:24]
        except:
            credits=0
        if Name not in director_dict["Name"]:
            director_dict["Name"].append(Name)
            director_dict['DOB'].append(DOB)
            director_dict['Birthplace'].append(Birthplace)
            director_dict['wins'].append(wins)
            director_dict['credits'].append(credits)
    except:
        pass
 

100%|██████████| 1304/1304 [10:43<00:00,  3.48it/s]

CPU times: user 3min 40s, sys: 4.18 s, total: 3min 44s
Wall time: 10min 43s


In [27]:
import datetime 
directordf = pd.DataFrame(director_dict) ### fix 'XXXX-0-0' date issue
for i in range(len(directordf.DOB)):
    if directordf.DOB[i] == None:
        directordf.DOB[i] = '2015-11-23'
        print directordf.DOB[i]
    if directordf.DOB[i].split('-')[1] == '0' or directordf.DOB[i].split('-')[2] == '0':
        directordf.DOB[i] = directordf.DOB[i].split('-')[0]+'-1'+'-1'
        print directordf.DOB[i]
        
today = datetime.datetime(2015, 11, 23)
directordf['age'] = (today - directordf.DOB.apply(pd.datetools.parse)).values/np.timedelta64(1, 'D')/365.25

1970-1-1
1968-1-1
1965-1-1
1970-1-1
1976-1-1
1973-1-1
1967-1-1
1979-1-1
1961-1-1
1972-1-1
1968-1-1
1974-1-1
2015-11-23
1972-1-1
1957-1-1
1968-1-1
2015-11-23
1965-1-1
2015-11-23
1972-1-1
1966-1-1
2015-11-23
1973-1-1
1952-1-1
1963-1-1
2015-11-23
1959-1-1
1977-1-1
1964-1-1
1972-1-1
1971-1-1
1972-1-1
2015-11-23


In [28]:
#directordf.to_csv('directordf2015.csv', encoding='utf-8')

In [29]:
%%time
actorurl = {}
### return a dict, with key the movie id and values the urls for actors 
for t in tqdm(tts, leave=True):
    try:
        actorurl[t] = []
        l = []
        url = "http://www.imdb.com/title/{}/".format(t)
        t1970=requests.get(url)
        soup = BeautifulSoup(t1970.text, "html.parser")
        divs = soup.find('table').findAll('div', attrs = {'class':'txt-block', 'itemprop':'actors'})[0]
        for div in divs.find_all('a'):
            l.append(div.get('href'))
        actorurl[t] = l[:-1]
    except:
        pass

100%|██████████| 1304/1304 [12:30<00:00,  2.05it/s]

CPU times: user 4min 23s, sys: 5.71 s, total: 4min 28s
Wall time: 12min 30s


In [30]:
#fd=open("actorurl2015.json","w")
#json.dump(actorurl, fd)
#fd.close()

In [31]:
#with open("actorurl2015.json") as json_file:
#    actorurl = json.load(json_file)

In [32]:
%%time
actor_dict = {"Name":[], "DOB":[], "Birthplace":[], "wins":[], "credits":[]}

for k in tqdm(actorurl.keys(), leave=True):
    url_list = actorurl[k]
    for u in url_list:
        #print u
        try:
            url = 'http://www.imdb.com{}'.format(u)
            t1990=requests.get(url)
            soup=BeautifulSoup(t1990.text, "html.parser")
            divs=soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].findAll('div', attrs = {'id':'name-born-info'})[0]
            try:
                Name=soup.findAll('table', attrs = {'id':'name-overview-widget-layout'})[0].find('span',attrs={'class':'itemprop'}).get_text()
            except:
                Name=None
            try:
                DOB=divs.find('time').get('datetime')
            except:
                DOB=None
            try:
                Birthplace=divs.findAll('a')[-1].get_text()
            except:
                Birthdplace=None
            try:
                wins=soup.findAll('div', attrs = {'class':'article highlighted'})[0].find_all('span')[0].get_text().strip()
            except:
                wins=0
                
            try:
                c = soup.findAll('div', attrs = {'id':'filmo-head-actor'})[0]
                c = str(c.get_text()[20:].strip())
                credit = float(''.join(x for x in c if x.isdigit()))
            except:
                c = soup.findAll('div', attrs = {'id':'filmo-head-actress'})[0]
                c = str(c.get_text()[20:].strip())
                credit = float(''.join(x for x in c if x.isdigit()))
                
            if Name not in actor_dict["Name"]:
                actor_dict['Name'].append(Name)
                actor_dict['DOB'].append(DOB)
                actor_dict['Birthplace'].append(Birthplace)
                actor_dict['wins'].append(wins)
                actor_dict['credits'].append(credit)
        except:
            pass

import datetime
today = datetime.datetime(2015, 11, 23)
actordf=pd.DataFrame(actor_dict)

for i in range(len(actordf.DOB)):
    if actordf.DOB[i] == None:
        actordf.DOB[i] = '2015-11-23'
        print actordf.DOB[i]
    if actordf.DOB[i].split('-')[1] == '0' or actordf.DOB[i].split('-')[2] == '0':
        actordf.DOB[i] = actordf.DOB[i].split('-')[0]+'-1'+'-1'
        print actordf.DOB[i]
    
### calculate the age
actordf['age'] = (today - actordf.DOB.apply(pd.datetools.parse)).values/np.timedelta64(1, 'D')/365.25
#actordf.to_csv('actordf2015.csv', encoding='utf-8')

100%|██████████| 304/304 [11:03<00:00,  0.55it/s]
/Users/rachelfyy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2015-11-23
1963-1-1
1993-1-1
1990-1-1
2015-11-23
1968-1-1
1947-1-1
2015-11-23
2015-11-23
2015-11-23
2015-11-23
1986-1-1
2015-11-23
2015-11-23
1963-1-1
2015-11-23
2015-11-23
2015-11-23
2015-11-23
CPU times: user 3min 19s, sys: 3.24 s, total: 3min 22s
Wall time: 11min 5s


/Users/rachelfyy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### We then merge actors and directors information into the movie dataframe.

In [48]:
#actors = pd.read_csv("actordf2015.csv", usecols=['Birthplace', 'DOB', 'Name', 'credits', 'wins', 'age'])

#directors = pd.read_csv("directordf2015.csv", usecols=['Birthplace', 'DOB', 'Name', 'credits', 'wins', 'age'])

#movies = pd.read_csv("movies2015.csv")
actors = actordf
directors = directordf
actors.rename(columns={'Birthplace': 'A_birthplace', 'DOB': 'A_DOB', 'Name': 'A_name', 'credits': 'A_credits', 'wins': 'A_wins',
                       'age': 'A_age' }, inplace=True)
directors.rename(columns={'Birthplace': 'D_birthplace', 'DOB': 'D_DOB','Name': 'D_name', 'credits': 'D_credits', 'wins': 'D_wins',
                       'age': 'D_age' }, inplace=True)
movies = movies.reset_index()

In [49]:
merge1 = pd.merge(movies, directors, left_on='Director', right_on='D_name', how='left')
merge2 = pd.merge(merge1, actors, left_on='Actors', right_on='A_name', how='left')
merge2.dropna(subset=['A_name'], how='all', inplace=True)
merge2.dropna(subset=['D_name'], how='all', inplace=True)
merge2.head()
#merge2 = merge2.drop(['Director', 'Actors', 'movie_id'], axis=1)


,imdbID,Title,Released,Awards,Director,Actors,Country,Language,Rated,Runtime,Year,Metascore,imdbRating,imdbVotes,tomatoConsensus,tomatoFresh,tomatoMeter,tomatoRating,tomatoReviews,tomatoRotten,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,Nominated Oscars,Won Oscars,Other wins,Other nominations,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Musical,Mystery,N/A,Romance,Sci-Fi,Sport,Thriller,War,Western,D_birthplace,D_DOB,D_name,D_credits,D_wins,D_age,A_birthplace,A_DOB,A_name,A_credits,A_wins,A_age
0,tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Chris Pratt,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,"San Francisco, California, USA",1976-9-13,Colin Trevorrow,7,6 nominations.,39.192334,"Virginia, Minnesota, USA",1979-6-21,Chris Pratt,43,8 wins & 21 nominations.,36.424367
1,tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Irrfan Khan,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,"San Francisco, California, USA",1976-9-13,Colin Trevorrow,7,6 nominations.,39.192334,"Jaipur, Rajasthan, India",1967-1-7,Irrfan Khan,130,15 wins & 19 nominations.,48.876112
2,tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Vincent D'Onofrio,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,"San Francisco, California, USA",1976-9-13,Colin Trevorrow,7,6 nominations.,39.192334,"Brooklyn, New York City, New York, USA",1959-6-30,Vincent D'Onofrio,89,Nominated for\n 1\n Primetime Emmy.,56.399726
3,tt0369610,Jurassic World,2015-06-12 00:00:00,1 win & 5 nominations.,Colin Trevorrow,Bryce Dallas Howard,USA,English,PG-13,124,2015,59,7.2,"314,481",Jurassic World can't match the original for sh...,194,71,6.7,274,80,80,3.9,206851,0,0,0,0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,"San Francisco, California, USA",1976-9-13,Colin Trevorrow,7,6 nominations.,39.192334,"Los Angeles, California, USA",1981-3-2,Bryce Dallas Howard,30,Nominated for\n 1\n Golden Globe.,34.726899
4,tt0470752,Ex Machina,2015-04-24 00:00:00,2 wins & 7 nominations.,Alex Garland,Domhnall Gleeson,UK,English,R,108,2015,78,7.7,"194,256",Ex Machina leans heavier on ideas than effects...,199,92,8.0,216,17,85,4.0,61929,0,0,0,0,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,"London, England, UK",1970-1-1,Alex Garland,2,4 wins & 10 nominations.,45.891855,"Dublin, Ireland",1983-5-12,Domhnall Gleeson,32,5 wins & 5 nominations.,32.533881


In [50]:
def numNomiOscar(s):
    re1='(Nominated)'
    re2='.*?'
    re3='(\\d+)'
    re4='.*?'
    re5='(Oscar)'
    re6='.*?'

    rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
    
    m = rg.search(s)
    if m:
        return m.group(2)
    else:
        return 0

def numWonOscar(s):
    re1='(Won)'
    re2='.*?'
    re3='(\\d+)'
    re4='.*?'
    re5='(Oscar)'
    re6='.*?'
    
    rg = re.compile(re1+re2+re3+re4+re5+re6,re.IGNORECASE|re.DOTALL)
    m = rg.search(s)
    if m:
        return m.group(2)
    else:
        return 0

In [51]:
merge2['Director Nominated Oscars'] = merge2.apply(lambda r: numNomiOscar(str(r.D_wins)), axis=1)
merge2['Director Won Oscars'] = merge2.apply(lambda r: numWonOscar(str(r.D_wins)), axis=1)
merge2['Actor Nominated Oscars'] = merge2.apply(lambda r: numNomiOscar(str(r.A_wins)), axis=1)
merge2['Actor Won Oscars'] = merge2.apply(lambda r: numWonOscar(str(r.A_wins)), axis=1)

In [52]:
print merge2.shape
print "sample size:", len(list(set(merge2.Title)))


(418, 65)
sample size: 169


### We then read in boxoffice and budget information that we scraped from wikipedia and adjust for different currecy units and inflation rate using historic CPI.

In [53]:
wiki = pd.read_csv('2015_movies_wiki.csv')
cpi = pd.read_csv('cpi.csv', usecols=['Year', 'Avg'])
wiki.columns

Index([u'Unnamed: 0', u'titles_new', u'wiki_url_new', u'imdb_link_new', u'rt_link_new', u'imdb_id', u'budget_new', u'currency_budget', u'box_office_new', u'currency_bo'], dtype='object')

In [54]:
#merge2
outcome = pd.merge(merge2, wiki, left_on='imdbID', right_on='imdb_id', how='left')

In [ ]:
import requests
import json
import numbers

def convertCurrency(C_from, amount, yr):
    ref_cpi = float(cpi[cpi.Year==yr]['Avg'])
    cur_cpi = float(cpi[cpi.Year==2015]['Avg'])
    try:
        if isinstance(amount, numbers.Number):
            url = ('https://currency-api.appspot.com/api/%s/%s.json') % (C_from, 'USD')
            temp = json.loads(requests.get(url).text)
            return amount * float(temp['rate']) * cur_cpi / ref_cpi ## adjust for exchange rate and inflation
        else:
            return None
    except: 
        print (C_from, amount)
        return(C_from)
        pass

outcome['budget_USD'] = outcome.apply(lambda r: convertCurrency(r.currency_budget, r.budget_new, r.Year), axis=1)
outcome['box_USD'] = outcome.apply(lambda r: convertCurrency(r.currency_bo, r.box_office_new, r.Year), axis=1)

In [42]:
print len(list(set(outcome[outcome.budget_USD.isnull()].imdbID)))
print len(list(set(outcome[outcome.box_USD.isnull()].imdbID)))

#cols =  outcome.columns
#print cols
outcome_copy = outcome
outcome = outcome.drop([ u'Unnamed: 0', u'titles_new', u'imdb_id',
                        u'budget_new', u'currency_budget', u'box_office_new', u'currency_bo'], axis=1)

168
70


In [43]:
outcome.shape

(422, 70)

In [44]:
outcome.to_csv('merge2015.csv', encoding='utf-8')